# Analyzing Used Car Listings on eBay Kleinanzeigen

In this project, the dataset used is from the German eBay website eBay Kleinanzeigen.
The dataset was originally scraped and uploaded to Kaggle. This dataset has sample of 50000 rows.
Data dictionary:
    
dateCrawled - When this ad was first crawled. All field-values are taken from this date.
name - Name of the car.
seller - Whether the seller is private or a dealer.
offerType - The type of listing
price - The price on the ad to sell the car.
abtest - Whether the listing is included in an A/B test.
vehicleType - The vehicle Type.
yearOfRegistration - The year in which which year the car was first registered.
gearbox - The transmission type.
powerPS - The power of the car in PS.
model - The car model name.
kilometer - How many kilometers the car has driven.
monthOfRegistration - The month in which which year the car was first registered.
fuelType - What type of fuel the car uses.
brand - The brand of the car.
notRepairedDamage - If the car has a damage which is not yet repaired.
dateCreated - The date on which the eBay listing was created.
nrOfPictures - The number of pictures in the ad.
postalCode - The postal code for the location of the vehicle.
lastSeenOnline - When the crawler saw this ad last online.

In [2]:
import pandas as pd
import numpy as np

In [3]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')


In [4]:
autos.head()
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

This dataset has 20 columns, 5 integer type and 15 string type. 5 columns have null values. Firstly, we clean the dataset and then perform data analysis

In [5]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

# Clean Columnnames

In [6]:
cols=autos.columns
new_cols=[]
for col in cols:
    col=col.lower()
    new_cols.append(col)
    
autos.columns =new_cols

autos.columns

Index(['datecrawled', 'name', 'seller', 'offertype', 'price', 'abtest',
       'vehicletype', 'yearofregistration', 'gearbox', 'powerps', 'model',
       'odometer', 'monthofregistration', 'fueltype', 'brand',
       'notrepaireddamage', 'datecreated', 'nrofpictures', 'postalcode',
       'lastseen'],
      dtype='object')

In [7]:
autos.rename({'datecrawled':'date_crawled','vehicletype':'vehicle_type','offertype':'offer_type','abtest':'ab_test',
              'yearofregistration':'registration_year','powerps':'power_ps','monthofregistration':'registration_month',
               'fueltype':'fuel_type','notrepaireddamage':'unrepaired_damage','datecreated':'date_created',
             'nrofpictures':'num_pics','postalcode':'postal_code','lastseen':'last_seen'},inplace=True,axis=1)

In [8]:
autos.rename({'num_':'num_pics'}, inplace=True, axis=1)

# Initial investigation of data

In [9]:
# For string data
autos.describe(include='O')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,gearbox,model,odometer,fuel_type,brand,unrepaired_damage,date_created,last_seen
count,50000,50000,50000,50000,50000,50000,44905,47320,47242,50000,45518,50000,40171,50000,50000
unique,48213,38754,2,2,2357,2,8,2,245,13,7,40,2,76,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,manuell,golf,"150,000km",benzin,volkswagen,nein,2016-04-03 00:00:00,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,36993,4024,32424,30107,10687,35232,1946,8


In [10]:
#For numeric data
autos.describe()

,registration_year,power_ps,registration_month,num_pics,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


From above describe commands, we can do changes in some of the columns such as odometer, seller, registration_year, price, num_pics, postal_code and others.

In [11]:
# checking the columns that may be dropped as they dont add value for analysis
print(autos["num_pics"].value_counts())
print(autos["seller"].value_counts())
print(autos["offer_type"].value_counts())

0    50000
Name: num_pics, dtype: int64
privat        49999
gewerblich        1
Name: seller, dtype: int64
Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64


In [12]:
# dropping above columns
autos=autos.drop(["num_pics","seller","offer_type"], axis=1)


In [13]:
# the price and odometers columns are in text, they can be converted to numeric 
autos["price"]=autos["price"].str.replace("$","").str.replace(",","").astype(int)
                          

In [14]:
autos["odometer"]=autos["odometer"].str.replace("km","").str.replace(",","").astype(int)


In [15]:
autos.rename({"odometer": "odometer_km"}, axis=1, inplace=True)


#Exploring odometer and price columns

In [16]:
autos["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

We can see that there are more vehicles with high odometer values. 

In [17]:
sort_price=autos["price"].value_counts().sort_index(ascending=False)

In [18]:
print(sort_price.head(20))
print(sort_price.tail(20))

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64
35       1
30       7
29       1
25       5
20       4
18       1
17       3
15       2
14       1
13       2
12       3
11       2
10       7
9        1
8        1
5        2
3        1
2        3
1      156
0     1421
Name: price, dtype: int64


There are outliers in price column starting from 999990, as the change is higher from its previous value 350000. The price of $1 can be kept as it can be starting bid price


In [19]:
autos=autos[autos["price"].between(1,350000)]

# Exploring date columns
- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `date_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website

In [20]:
autos[['date_crawled','date_created','last_seen','registration_month','registration_year']][0:5]

,date_crawled,date_created,last_seen,registration_month,registration_year
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54,3,2004
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08,6,1997
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37,7,2009
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28,6,2007
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50,7,2003


The columns date_crawled, date_created and last_seen are stored as strings. We can convert these to numeric type

In [21]:
#counting date occurences as number
(autos["date_crawled"]
        .str[:10]
        .value_counts(dropna=False)
        .sort_index()
        )

2016-03-05    1230
2016-03-06     682
2016-03-07    1749
2016-03-08    1617
2016-03-09    1607
2016-03-10    1563
2016-03-11    1582
2016-03-12    1793
2016-03-13     761
2016-03-14    1775
2016-03-15    1665
2016-03-16    1438
2016-03-17    1536
2016-03-18     627
2016-03-19    1689
2016-03-20    1840
2016-03-21    1815
2016-03-22    1602
2016-03-23    1565
2016-03-24    1425
2016-03-25    1535
2016-03-26    1564
2016-03-27    1510
2016-03-28    1693
2016-03-29    1656
2016-03-30    1636
2016-03-31    1546
2016-04-01    1636
2016-04-02    1723
2016-04-03    1875
2016-04-04    1772
2016-04-05     636
2016-04-06     154
2016-04-07      68
Name: date_crawled, dtype: int64

In [22]:
(autos["date_crawled"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_values()
        )

2016-04-07    0.001400
2016-04-06    0.003171
2016-03-18    0.012911
2016-04-05    0.013096
2016-03-06    0.014043
2016-03-13    0.015670
2016-03-05    0.025327
2016-03-24    0.029342
2016-03-16    0.029610
2016-03-27    0.031092
2016-03-25    0.031607
2016-03-17    0.031628
2016-03-31    0.031834
2016-03-10    0.032184
2016-03-26    0.032204
2016-03-23    0.032225
2016-03-11    0.032575
2016-03-22    0.032987
2016-03-09    0.033090
2016-03-08    0.033296
2016-04-01    0.033687
2016-03-30    0.033687
2016-03-29    0.034099
2016-03-15    0.034284
2016-03-19    0.034778
2016-03-28    0.034860
2016-04-02    0.035478
2016-03-07    0.036014
2016-04-04    0.036487
2016-03-14    0.036549
2016-03-12    0.036920
2016-03-21    0.037373
2016-03-20    0.037887
2016-04-03    0.038608
Name: date_crawled, dtype: float64

It can be seen that the distribution of listings crawled in one month period between Marcha and April on each day is almost same.

In [23]:
(autos["date_created"]
        .str[:10]
        .value_counts(normalize=True, dropna=False)
        .sort_index()
        )

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: date_created, Length: 76, dtype: float64

Most of the ads are created with in 1-2 months of the listing date barring some where the listing date was at around 9 months.


In [24]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The minimum with 1000 and maximum 9999 for registration years look incorrect. We will consider the registration years between 1900 and 2016.


In [25]:
autos = autos[autos["registration_year"].between(1900,2016)]
autos["registration_year"].value_counts(normalize=True)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
          ...   
1938    0.000021
1948    0.000021
1927    0.000021
1931    0.000021
1952    0.000021
Name: registration_year, Length: 78, dtype: float64

# Exploring price by brands

In [26]:
autos["brand"].value_counts(normalize=True)


volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

Volkswagen is the most popular brand, with approximately equal to the  combined sale of the next two brands.

In [36]:
brand_counts= autos["brand"].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .05].index
print(common_brands)


Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [64]:
brand_mean_price={}
for brands in common_brands:
    req_brand=autos["brand"]==brands
    mean_price=autos[req_brand]["price"].mean()
    brand_mean_price[brands]= int(mean_price)
    
brand_mean_price   

{'volkswagen': 5402,
 'bmw': 8332,
 'opel': 2975,
 'mercedes_benz': 8628,
 'audi': 9336,
 'ford': 3749}

In [66]:
brand_mean_price_series = pd.Series(brand_mean_price)
brand_mean_price_df=pd.DataFrame(brand_mean_price_series, columns=["mean_price"])
print(brand_mean_price_df)

               mean_price
volkswagen           5402
bmw                  8332
opel                 2975
mercedes_benz        8628
audi                 9336
ford                 3749


When we look at the mean_price for brands, there is a higher price gap among them:

Audi, BMW and Mercedes Benz are more expensive
Ford and Opel are less expensive
Most popular Volkswagen brand's price is in middle - this can be viewed as people with income between high and low opting for this brand's vehicles

# Exploring Mileage


In [59]:
brand_mean_mileage = {}
for brands in common_brands:
    req_brand = autos["brand"] == brands
    mean_mileage = autos[req_brand]["odometer_km"].mean()
    brand_mean_mileage[brands] = int(mean_mileage)

mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_price).sort_values(ascending=False)

In [68]:
brand_mean_mileage = pd.DataFrame(mean_mileage,columns=['mean_mileage'])

In [69]:
print(brand_mean_mileage)

               mean_mileage
bmw                  132572
mercedes_benz        130788
opel                 129310
audi                 129157
volkswagen           128707
ford                 124266


In [70]:
brand_info=brand_mean_mileage

In [71]:
brand_info["mean_price"]=mean_prices

In [72]:
brand_info

,mean_mileage,mean_price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129310,2975
audi,129157,9336
volkswagen,128707,5402
ford,124266,3749


The more expensive vehicles have higher mileage when compared to the less expensive vehicles. The % difference in mean_mileage is less when compared to mean_prices among high and low expensive vehicles.